# Street View Hyperlapse Automation 🎬

**Created:** September 23, 2025  
**Purpose:** Automate the capture of Google Street View images for creating hyperlapse videos

This notebook demonstrates how to use Playwright to automatically navigate through Street View and capture sequential frames for hyperlapse creation.

## Key Features:
- 🗺️ Navigate to any Street View location
- 📸 Capture sequential frames using natural forward movement
- 🎯 Center-click navigation for smooth progression
- 📊 Interactive configuration and results display
- 🎥 Ready for video creation with FFmpeg

---


## 1. Install Dependencies

First, let's ensure we have all required packages:


In [28]:
# Install required packages if not already installed
import subprocess
import sys

def install_package(package):
    try:
        __import__(package)
        print(f"✅ {package} is already installed")
    except ImportError:
        print(f"📦 Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Check and install packages
install_package("playwright")

# Install Playwright browsers
print("🌐 Installing Playwright browsers...")
subprocess.run([sys.executable, "-m", "playwright", "install", "chromium"], check=True)
print("✅ Setup complete!")


✅ playwright is already installed
🌐 Installing Playwright browsers...
✅ Setup complete!


## 2. Import Libraries and Setup


In [29]:
import asyncio
import os
import time
from datetime import datetime
from pathlib import Path
from playwright.async_api import async_playwright
from IPython.display import Image, display, Markdown

# Install matplotlib if not present
install_package("matplotlib")

import matplotlib.pyplot as plt
from matplotlib.image import imread

print("📚 All libraries imported successfully!")


✅ matplotlib is already installed
📚 All libraries imported successfully!


## 3. Configuration

Configure your capture settings here:


In [ ]:
# Generate unique folder for this capture session
session_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
unique_output_dir = f"frames/capture_{session_timestamp}"

# Configuration Settings
CONFIG = {
    # Example Street View URLs - uncomment one or add your own
    "street_view_url": "https://www.google.com/maps/@45.7118968,-121.7791618,3a,75y,229.87h,90.44t/data=!3m7!1e1!3m5!1saegp4MZFE9ahTlhjRA1jrQ!2e0!6shttps:%2F%2Fstreetviewpixels-pa.googleapis.com%2Fv1%2Fthumbnail%3Fcb_client%3Dmaps_sv.tactile%26w%3D900%26h%3D600%26pitch%3D-0.43594011637384256%26panoid%3Daegp4MZFE9ahTlhjRA1jrQ%26yaw%3D229.86665737018365!7i16384!8i8192?entry=ttu&g_ep=EgoyMDI1MDkxNy4wIKXMDSoASAFQAw%3D%3D",
    # Alternative URLs to try:
    # "street_view_url": "https://www.google.com/maps/@40.7579787,-73.9881229,3a,75y,69.92h,90t/data=!3m8!1e1!3m6!1sCIHM0ogKEICAgIDEsaTtgQE!2e10!3e11",  # Times Square NYC
    
    "num_frames": 25,                    # Number of frames to capture
    "output_dir": unique_output_dir,     # Unique timestamped directory
    "base_name": "frame",                # Base name for frame files
    "navigation_direction": "right",   # Navigation: "forward" (⬆️ arrow), "left" (⬅️ click), "right" (➡️ click)
    "delay_seconds": 1.5,                # Delay between captures
    "headless": False,                   # Set True to hide browser window
    "viewport_width": 1920,              # Browser width
    "viewport_height": 1080              # Browser height
}

print("⚙️ Configuration loaded:")
print(f"  📁 Session folder: {unique_output_dir}")
print("  " + "=" * 40)
for key, value in CONFIG.items():
    if key == "street_view_url":
        print(f"  📍 {key}: {value[:50]}...")
    elif key == "output_dir":
        print(f"  📁 {key}: {value}")
    elif key == "navigation_direction":
        direction_emoji = {"forward": "⬆️", "left": "⬅️", "right": "➡️"}.get(value, "🧭")
        print(f"  🧭 {key}: {value} {direction_emoji}")
    else:
        print(f"  🔧 {key}: {value}")


⚙️ Configuration loaded:
  📁 Session folder: frames/capture_20250923_083423
  📍 street_view_url: https://www.google.com/maps/@45.7163833,-121.79325...
  🔧 num_frames: 25
  📁 output_dir: frames/capture_20250923_083423
  🔧 base_name: frame
  🧭 navigation_direction: left ⬅️
  🔧 delay_seconds: 1.5
  🔧 headless: False
  🔧 viewport_width: 1920
  🔧 viewport_height: 1080


## 4. Street View Hyperlapse Class

**📁 Organized Captures**: Each time you run this notebook, it creates a unique timestamped folder for your frames and videos, keeping all your captures perfectly organized!

**🧭 Hybrid Navigation System**: Intelligent navigation based on direction:
- **⬆️ Forward** (`"forward"`): Up-arrow key for smooth movement along street paths
- **⬅️ Left** (`"left"`): Click left side of Street View for natural counter-clockwise panning
- **➡️ Right** (`"right"`): Click right side of Street View for natural clockwise panning

**💡 Creative Ideas**:
- **Street Hyperlapse**: Use `"forward"` to travel down a scenic road or bridge
- **Panoramic Sweep**: Use `"left"` or `"right"` to show a 360° view of landmarks
- **Architecture Tour**: Use `"left"/"right"` to sweep around buildings or monuments

**🎯 Why Hybrid Navigation?**
- **Forward**: Arrow keys provide consistent street-to-street movement
- **Left/Right**: Side clicking mimics natural mouse panning for smoother rotation

Our main automation class with all the functionality:


In [31]:
class StreetViewHyperlapse:
    def __init__(self, config):
        """Initialize with configuration settings."""
        self.config = config
        self.output_dir = Path(config["output_dir"])
        self.browser = None
        self.page = None
        self.captured_frames = []
        
        # Create output directory
        self.output_dir.mkdir(parents=True, exist_ok=True)
        print(f"📁 Output directory ready: {self.output_dir}")
    
    def create_location_html(self):
        """Create a simple HTML file with the Street View URL link."""
        print("🌐 Creating location HTML file...")
        
        # Extract coordinates and info from URL if possible
        url = self.config["street_view_url"]
        session_name = self.output_dir.name
        timestamp = datetime.now().strftime("%B %d, %Y at %H:%M:%S")
        
        html_content = f'''<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Street View Location - {session_name}</title>
    <style>
        body {{
            font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', system-ui, sans-serif;
            max-width: 800px;
            margin: 40px auto;
            padding: 20px;
            background: #f8f9fa;
        }}
        .container {{
            background: white;
            padding: 30px;
            border-radius: 12px;
            box-shadow: 0 4px 12px rgba(0,0,0,0.1);
        }}
        h1 {{
            color: #2c3e50;
            text-align: center;
            margin-bottom: 30px;
        }}
        .info-card {{
            background: #e3f2fd;
            padding: 20px;
            border-radius: 8px;
            margin: 20px 0;
            border-left: 4px solid #2196f3;
        }}
        .street-view-link {{
            background: #4285f4;
            color: white;
            padding: 15px 30px;
            border-radius: 6px;
            text-decoration: none;
            display: inline-block;
            font-weight: 500;
            transition: background 0.3s;
            margin: 20px 0;
        }}
        .street-view-link:hover {{
            background: #3367d6;
        }}
        .settings {{
            background: #f5f5f5;
            padding: 15px;
            border-radius: 6px;
            margin-top: 20px;
        }}
        .settings h3 {{
            margin-top: 0;
            color: #666;
        }}
        .emoji {{
            font-style: normal;
            font-size: 1.2em;
        }}
    </style>
</head>
<body>
    <div class="container">
        <h1><span class="emoji">📍</span> Street View Hyperlapse Location</h1>
        
        <div class="info-card">
            <h3><span class="emoji">📊</span> Capture Session Info</h3>
            <p><strong>Session:</strong> {session_name}</p>
            <p><strong>Created:</strong> {timestamp}</p>
            <p><strong>Navigation:</strong> {self.config.get("navigation_direction", "forward").title()}</p>
            <p><strong>Frames to capture:</strong> {self.config.get("num_frames", "N/A")}</p>
        </div>
        
        <div style="text-align: center;">
            <a href="{url}" target="_blank" class="street-view-link">
                <span class="emoji">🗺️</span> Open in Google Street View
            </a>
        </div>
        
        <div class="settings">
            <h3><span class="emoji">⚙️</span> Capture Settings</h3>
            <p><strong>Direction:</strong> {self.config.get("navigation_direction", "forward")}</p>
            <p><strong>Delay:</strong> {self.config.get("delay_seconds", "N/A")} seconds</p>
            <p><strong>Resolution:</strong> {self.config.get("viewport_width", "N/A")}x{self.config.get("viewport_height", "N/A")}</p>
        </div>
        
        <div style="text-align: center; margin-top: 30px; color: #666; font-size: 0.9em;">
            <p>Generated by Street View Hyperlapse Automation</p>
        </div>
    </div>
</body>
</html>'''
        
        # Save HTML file
        html_file = self.output_dir / "location.html"
        with open(html_file, 'w', encoding='utf-8') as f:
            f.write(html_content)
        
        print(f"✅ Location HTML created: {html_file}")
        return str(html_file)
        
    async def setup_browser(self):
        """Setup Playwright browser and page."""
        print("🚀 Setting up browser...")
        
        self.playwright = await async_playwright().start()
        self.browser = await self.playwright.chromium.launch(
            headless=self.config["headless"]
        )
        self.page = await self.browser.new_page()
        
        # Set viewport size for better image quality
        await self.page.set_viewport_size({
            "width": self.config["viewport_width"], 
            "height": self.config["viewport_height"]
        })
        
        print("✅ Browser setup complete")
        
    async def navigate_to_location(self):
        """Navigate to the Street View location."""
        print(f"🗺️ Navigating to Street View location...")
        
        await self.page.goto(self.config["street_view_url"])
        
        # Wait for Street View to load
        try:
            await self.page.wait_for_selector(
                '[role="application"][aria-label*="Street View"]', 
                timeout=10000
            )
            await asyncio.sleep(3)  # Additional loading time
            print("✅ Street View loaded successfully")
        except Exception as e:
            print(f"⚠️ Warning: {e}")
            await asyncio.sleep(5)  # Fallback wait time
            
    async def capture_frame(self, frame_number):
        """Capture a single frame."""
        timestamp = datetime.now().strftime("%H%M%S")
        filename = f"{self.config['base_name']}_{frame_number:03d}_{timestamp}.png"
        filepath = self.output_dir / filename
        
        print(f"📸 Capturing frame {frame_number}: {filename}")
        
        # Take screenshot
        await self.page.screenshot(
            path=str(filepath),
            full_page=False,
            type='png'
        )
        
        self.captured_frames.append(str(filepath))
        return str(filepath)
        
    async def advance_street_view(self):
        """Navigate in Street View using configured direction (forward, left, right)."""
        try:
            # Find the Street View application area
            street_view_app = await self.page.wait_for_selector(
                '[role="application"][aria-label*="Street View"]',
                timeout=5000
            )
            
            if street_view_app:
                direction = self.config["navigation_direction"]
                
                if direction == "forward":
                    # Forward movement: use up arrow key
                    print("⬆️ Moving forward in Street View (up-arrow)...")
                    await street_view_app.click()  # Ensure focus
                    await asyncio.sleep(0.5)
                    await self.page.keyboard.press('ArrowUp')
                    
                elif direction == "left":
                    # Left movement: click on left side of Street View
                    print("⬅️ Panning left in Street View (left-side click)...")
                    box = await street_view_app.bounding_box()
                    if box:
                        # Click on left third of the Street View area
                        left_x = box['x'] + (box['width'] * 0.25)
                        center_y = box['y'] + (box['height'] * 0.5)
                        await self.page.mouse.click(left_x, center_y)
                    else:
                        print("⚠️ Could not get Street View bounds, using center-left click")
                        await street_view_app.click()
                        
                elif direction == "right":
                    # Right movement: click on right side of Street View
                    print("➡️ Panning right in Street View (right-side click)...")
                    box = await street_view_app.bounding_box()
                    if box:
                        # Click on right third of the Street View area
                        right_x = box['x'] + (box['width'] * 0.75)
                        center_y = box['y'] + (box['height'] * 0.5)
                        await self.page.mouse.click(right_x, center_y)
                    else:
                        print("⚠️ Could not get Street View bounds, using center-right click")
                        await street_view_app.click()
                        
                else:
                    print(f"⚠️ Invalid direction '{direction}'. Using 'forward' as default.")
                    await street_view_app.click()  # Ensure focus
                    await asyncio.sleep(0.5)
                    await self.page.keyboard.press('ArrowUp')
                
                await asyncio.sleep(self.config["delay_seconds"])
                return True
            else:
                print("⚠️ Could not find Street View application area")
                return False
                
        except Exception as e:
            print(f"❌ Error navigating Street View: {e}")
            return False
            
    async def capture_sequence(self):
        """Capture the complete hyperlapse sequence."""
        print(f"🎬 Starting capture sequence: {self.config['num_frames']} frames")
        print("=" * 50)
        
        # Create HTML file with location info at the start
        self.create_location_html()
        
        # Navigate to location
        await self.navigate_to_location()
        
        # Capture first frame
        await self.capture_frame(1)
        
        # Capture subsequent frames
        for frame_num in range(2, self.config['num_frames'] + 1):
            # Advance to next position
            success = await self.advance_street_view()
            
            if not success:
                print(f"⚠️ Could not advance further. Stopping at frame {frame_num - 1}")
                break
                
            # Capture frame
            await self.capture_frame(frame_num)
            print(f"✅ Progress: {frame_num}/{self.config['num_frames']}")
            
        print("=" * 50)
        print(f"🎉 Capture complete! {len(self.captured_frames)} frames captured")
        
    async def close_browser(self):
        """Clean up browser resources."""
        if self.browser:
            await self.browser.close()
        if hasattr(self, 'playwright'):
            await self.playwright.stop()
        print("🔄 Browser closed")

print("✅ StreetViewHyperlapse class defined")


✅ StreetViewHyperlapse class defined


## 5. Run the Capture Process

Now let's execute the capture automation:


In [32]:
# Initialize the hyperlapse capture system
hyperlapse = StreetViewHyperlapse(CONFIG)

async def run_capture():
    """Run the complete capture process."""
    try:
        # Setup browser
        await hyperlapse.setup_browser()
        
        # Capture sequence
        await hyperlapse.capture_sequence()
        
    except KeyboardInterrupt:
        print("\n⚠️ Capture interrupted by user")
    except Exception as e:
        print(f"\n❌ Error: {e}")
    finally:
        # Clean up
        await hyperlapse.close_browser()

# Run the capture process
print("🎬 Starting Street View Hyperlapse capture...")
await run_capture()


📁 Output directory ready: frames/capture_20250923_083423
🎬 Starting Street View Hyperlapse capture...
🚀 Setting up browser...
✅ Browser setup complete
🎬 Starting capture sequence: 25 frames
🌐 Creating location HTML file...
✅ Location HTML created: frames/capture_20250923_083423/location.html
🗺️ Navigating to Street View location...
✅ Street View loaded successfully
📸 Capturing frame 1: frame_001_083428.png
⬅️ Panning left in Street View (left-side click)...
📸 Capturing frame 2: frame_002_083430.png
✅ Progress: 2/25
⬅️ Panning left in Street View (left-side click)...
📸 Capturing frame 3: frame_003_083432.png
✅ Progress: 3/25
⬅️ Panning left in Street View (left-side click)...
📸 Capturing frame 4: frame_004_083433.png
✅ Progress: 4/25
⬅️ Panning left in Street View (left-side click)...
📸 Capturing frame 5: frame_005_083435.png
✅ Progress: 5/25
⬅️ Panning left in Street View (left-side click)...
📸 Capturing frame 6: frame_006_083437.png
✅ Progress: 6/25
⬅️ Panning left in Street View (lef

## 6. Display Captured Frames

View the frames we captured inline:


## 7. Create Video with FFmpeg (Optional)

If you have FFmpeg installed, create a hyperlapse video:


In [33]:
def create_hyperlapse_video():
    """Create a hyperlapse video using FFmpeg (if available)."""
    if not hyperlapse.captured_frames:
        print("❌ No frames available for video creation.")
        return
        
    try:
        # Check if FFmpeg is available
        result = subprocess.run(['ffmpeg', '-version'], 
                              capture_output=True, text=True)
        if result.returncode != 0:
            raise FileNotFoundError("FFmpeg not found")
            
        print("🎬 FFmpeg found! Creating hyperlapse video...")
        
        # Create video filename
        video_filename = f"hyperlapse_{datetime.now().strftime('%Y%m%d_%H%M%S')}.mp4"
        video_path = hyperlapse.output_dir / video_filename
        
        # FFmpeg command
        cmd = [
            'ffmpeg', '-y',  # Overwrite output
            '-r', '2',  # Frame rate (2 FPS)
            '-pattern_type', 'glob',
            '-i', str(hyperlapse.output_dir / '*.png'),
            '-c:v', 'libx264',
            '-pix_fmt', 'yuv420p',
            '-vf', 'scale=1920:1080',
            str(video_path)
        ]
        
        print(f"🔧 Creating video: {video_filename}")
        result = subprocess.run(cmd, capture_output=True, text=True)
        
        if result.returncode == 0:
            print(f"✅ Video created successfully!")
            print(f"📁 Location: {video_path}")
            print(f"📊 Size: {video_path.stat().st_size / 1024 / 1024:.1f} MB")
        else:
            print(f"❌ Error: {result.stderr}")
            
    except FileNotFoundError:
        print("⚠️ FFmpeg not found.")
        print("   Install FFmpeg to create videos automatically.")
        print("   Manual command:")
        print(f"   ffmpeg -r 2 -pattern_type glob -i '{hyperlapse.output_dir}/*.png' -c:v libx264 -pix_fmt yuv420p hyperlapse.mp4")
        
# Create the video
create_hyperlapse_video()


🎬 FFmpeg found! Creating hyperlapse video...
🔧 Creating video: hyperlapse_20250923_083513.mp4
✅ Video created successfully!
📁 Location: frames/capture_20250923_083423/hyperlapse_20250923_083513.mp4
📊 Size: 8.6 MB


---

## 🎉 Street View Hyperlapse Complete!

### What We Accomplished:
- ✅ **Interactive Setup**: Automatic dependency installation
- ✅ **Smart Navigation**: Center-click method for smooth forward movement  
- ✅ **Visual Results**: Display captured frames inline
- ✅ **Video Creation**: Optional FFmpeg integration
- ✅ **Configurable**: Easy to adjust settings and locations

### Try Different Locations:
```python
# Update CONFIG["street_view_url"] with one of these:

# Golden Gate Bridge
"https://www.google.com/maps/@37.8199,-122.4783,3a,75y,320h,90t"

# Route 66, Arizona  
"https://www.google.com/maps/@35.2137,-111.6265,3a,75y,270h,90t"

# Lake Garda, Italy
"https://maps.app.goo.gl/6WU4y2NDYT6LzA127"

# Pacific Coast Highway
"https://www.google.com/maps/@36.2677,-121.8053,3a,75y,200h,90t"

# sideways Hood River
"https://www.google.com/maps/place/Hood+River,+OR+97031/@45.7294933,-121.5326758,3a,75y,197.03h,89.9t/data=!3m7!1e1!3m5!1s9AsF93D2P3m6yRnxU_P_TA!2e0!6shttps:%2F%2Fstreetviewpixels-pa.googleapis.com%2Fv1%2Fthumbnail%3Fcb_client%3Dmaps_sv.tactile%26w%3D900%26h%3D600%26pitch%3D0.09839617411907398%26panoid%3D9AsF93D2P3m6yRnxU_P_TA%26yaw%3D197.02538091596819!7i16384!8i8192!4m6!3m5!1s0x54960e614c1c0fe7:0x56f6cf7e4d7d985c!8m2!3d45.708857!4d-121.5122706!16zL20vMHpkM20?entry=ttu&g_ep=EgoyMDI1MDkxNy4wIKXMDSoASAFQAw%3D%3D"

```



### Pro Tips:
- 🎯 **Best Locations**: Bridges, highways, long straight roads
- 📈 **More Frames**: Try 10-20 frames for smoother videos
- 🎬 **Frame Rates**: 1-3 FPS creates dramatic time-lapse effect
- ⏱️ **Timing**: Adjust `delay_seconds` for better loading

### Video Creation:
```bash
# Create smooth hyperlapse at 3 FPS
ffmpeg -r 3 -pattern_type glob -i "frames/*.png" \
       -c:v libx264 -pix_fmt yuv420p hyperlapse.mp4

# Add motion blur for ultra-smooth effect
ffmpeg -r 5 -pattern_type glob -i "frames/*.png" \
       -vf "minterpolate=fps=30,scale=1920:1080" \
       hyperlapse_smooth.mp4
```

**Happy hyperlapsing! 🚗💨✨**


## 8. Python Video Creation Methods

Instead of FFmpeg, you can use Python libraries to create videos:


In [34]:
# Method 1: OpenCV (Most Popular) - Install: pip install opencv-python
def create_video_with_opencv(fps=2):
    """Create video using OpenCV - fastest and most reliable."""
    if not hyperlapse.captured_frames:
        print("❌ No frames to process")
        return
        
    try:
        import cv2
        import glob
        
        print("🎬 Creating video with OpenCV...")
        
        # Get frame files
        frame_files = sorted(glob.glob(str(hyperlapse.output_dir / "*.png")))
        
        # Read first frame for dimensions
        first_frame = cv2.imread(frame_files[0])
        height, width, layers = first_frame.shape
        
        # Create video writer
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        video_path = hyperlapse.output_dir / f"hyperlapse_opencv_{datetime.now().strftime('%H%M%S')}.mp4"
        video_writer = cv2.VideoWriter(str(video_path), fourcc, fps, (width, height))
        
        # Process frames
        for frame_file in frame_files:
            frame = cv2.imread(frame_file)
            video_writer.write(frame)
        
        video_writer.release()
        print(f"✅ Video created: {video_path}")
        return str(video_path)
        
    except ImportError:
        print("⚠️ OpenCV not installed. Install with: pip install opencv-python")
        return None

# Method 2: MoviePy (Best for Effects) - Install: pip install moviepy
def create_video_with_moviepy(fps=2):
    """Create video using MoviePy - great for effects and editing."""
    if not hyperlapse.captured_frames:
        print("❌ No frames to process")
        return
        
    try:
        from moviepy.editor import ImageSequenceClip
        
        print("🎭 Creating video with MoviePy...")
        
        # Create clip from frames
        clip = ImageSequenceClip(hyperlapse.captured_frames, fps=fps)
        
        # Write video
        video_path = hyperlapse.output_dir / f"hyperlapse_moviepy_{datetime.now().strftime('%H%M%S')}.mp4"
        clip.write_videofile(
            str(video_path),
            codec='libx264',
            audio=False,
            verbose=False,
            logger=None
        )
        
        print(f"✅ Video created: {video_path}")
        return str(video_path)
        
    except ImportError:
        print("⚠️ MoviePy not installed. Install with: pip install moviepy")
        return None

# Method 3: Create Animated GIF
def create_animated_gif(duration=500):
    """Create animated GIF using Pillow."""
    if not hyperlapse.captured_frames:
        print("❌ No frames to process")
        return
        
    try:
        from PIL import Image
        
        print("🎞️ Creating animated GIF...")
        
        # Load and resize images
        images = []
        for frame_path in hyperlapse.captured_frames:
            img = Image.open(frame_path)
            # Resize for smaller file
            img = img.resize((img.width // 2, img.height // 2), Image.Resampling.LANCZOS)
            images.append(img)
        
        # Save as GIF
        gif_path = hyperlapse.output_dir / f"hyperlapse_{datetime.now().strftime('%H%M%S')}.gif"
        images[0].save(
            str(gif_path),
            save_all=True,
            append_images=images[1:],
            duration=duration,
            loop=0
        )
        
        print(f"✅ Animated GIF created: {gif_path}")
        return str(gif_path)
        
    except ImportError:
        print("⚠️ Pillow not available (should be installed with matplotlib)")
        return None

print("📚 Video creation functions defined!")
print("🎬 Available methods:")
print("  • create_video_with_opencv(fps=2)")
print("  • create_video_with_moviepy(fps=2)") 
print("  • create_animated_gif(duration=500)")


📚 Video creation functions defined!
🎬 Available methods:
  • create_video_with_opencv(fps=2)
  • create_video_with_moviepy(fps=2)
  • create_animated_gif(duration=500)


In [35]:
# Try creating a video with your preferred method:

# Option 1: OpenCV (recommended for speed)
create_video_with_opencv(fps=3)

# Option 2: MoviePy (recommended for effects) 
# create_video_with_moviepy(fps=2)

# Option 3: Animated GIF (great for sharing)
# create_animated_gif(duration=400)

print("💡 Uncomment one of the lines above to create a video!")


⚠️ OpenCV not installed. Install with: pip install opencv-python
💡 Uncomment one of the lines above to create a video!
